In [2]:
from copy import deepcopy
import scipy
import tensorflow as tf
from graph_clustering import A_binarize, creating_label
from graph_features import graph_norm
import numpy as np
import pyedflib
import time
from preprocessing_functions import preprocess_data
import os
import pandas as pd

In [6]:
# We assume that the 3 loss function is defined and we include the new proposed decoder model
# Experiment stuff
subject_num = 149  # Size of dataset
run_num = 1  # Number of "experiences" of each task (this should be one)
num_electrodes = 60  # Number of EEG electrodes (this is excluding ["Iz", "I1", "I2", "Resp", "PO4", "PO3", "FT9", "Status"])
data_length = 60500  # Minimum data length, cut off anything outside this data length
BASE_PATH = r"C:\Users\timmy\Downloads\park-eeg"

In [7]:
# Returns the dataset and labels
def load_dataset():
    # Format is sub-NUM_task-Rest_eeg.edf
    data = []
    for subject in range(subject_num):
        subject_list = []
        for run in range(run_num):
            file_name = r"\sub-{}_task-Rest_eeg.edf".format(str(subject + 1).zfill(3))
            try:
                f = pyedflib.EdfReader(BASE_PATH + file_name)
            except Exception as e:
                print(e)
                continue

            electrode_list = []
            for electrode in range(num_electrodes):  # Electrodes are zero-indexed
                # Iz, I1, I2, PO4, PO3, FT9 is not present in all subjects... exclude
                # Status/Resp... I'm assuming is ground/reference electrode
                # Regardless, both status/resp are not present in all subjects
                if f.getLabel(electrode) in ("Iz", "I1", "I2", "Resp", "PO4", "PO3", "FT9", "Status"):
                    continue
                electrode_list.append(f.readSignal(electrode)[:data_length])
            subject_list.append(electrode_list)
            f._close()
            del f  # Don't read all files into memory

        data.append(subject_list)
        print("Subject", subject, "done!")

    raw_labels = pd.read_csv("participants.tsv", sep="\t")
    # Get rid of #68 and add labels
    new_data = []
    labels = []
    for index, subject in enumerate(data):

        if not subject:
            continue

        new_data.append(subject)

        query = "sub-" + str(index + 1).zfill(3)
        results = raw_labels[(raw_labels["participant_id"] == query)]
        labels.append(results.iloc[0]["GROUP"])

        print("Label", index, "done!")

    return np.array(new_data), np.array([i == "PD" for i in labels])

In [8]:
Binary = True
partial_subject = False
part_channel = False
verbose = True
nb_run = 5  # 5-fold cross validation
step = 160 * 0 + 80  # 1-window*alpha%
Fs = 500  # Sampling frequency (hz)
Ts = 1 / Fs  # Time in seconds ??

# Initialized numpy arrays for in-train data
accuracy = np.zeros((nb_run, 1))
accuracy2 = np.zeros((nb_run, 1))
Computational_time = np.zeros((nb_run, 1))
num_epoch = np.zeros((nb_run, 1))
full_time = np.zeros((nb_run, 1))
roc_auc = np.zeros((nb_run, 1))
EER = np.zeros((nb_run, 1))

x_raw_all, labels = load_dataset()  # import data for all subject 

loss_function = 3  # 3 loss function is defined
decoder_adj = True  # include new decoder model

FLAGS_features = False  # Whether or not to include pre-defined features
features_init_train = None
features_init_test = None

Subject 0 done!
Subject 1 done!
Subject 2 done!
Subject 3 done!
Subject 4 done!
Subject 5 done!
Subject 6 done!
Subject 7 done!
Subject 8 done!
Subject 9 done!
Subject 10 done!
Subject 11 done!
Subject 12 done!
Subject 13 done!
Subject 14 done!
Subject 15 done!
Subject 16 done!
Subject 17 done!
Subject 18 done!
Subject 19 done!
Subject 20 done!
Subject 21 done!
Subject 22 done!
Subject 23 done!
Subject 24 done!
Subject 25 done!
Subject 26 done!
Subject 27 done!
Subject 28 done!
Subject 29 done!
Subject 30 done!
Subject 31 done!
Subject 32 done!
Subject 33 done!
Subject 34 done!
Subject 35 done!
Subject 36 done!
Subject 37 done!
Subject 38 done!
Subject 39 done!
Subject 40 done!
Subject 41 done!
Subject 42 done!
Subject 43 done!
Subject 44 done!
Subject 45 done!
Subject 46 done!
Subject 47 done!
Subject 48 done!
Subject 49 done!
Subject 50 done!
Subject 51 done!
Subject 52 done!
Subject 53 done!
Subject 54 done!
Subject 55 done!
Subject 56 done!
Subject 57 done!
Subject 58 done!
Subject

In [9]:
def invlogit(z):  # convert decoded adjancy matrix to original space
    return 1 - 1 / (1 + np.exp(z))

def Adj_matrix(train_x, test_x):
    if (Binary):
        # Change weighted matrix to binary matrix with threshold
        percentile = 0.75
        adj_train = A_binarize(A_matrix=train_x, percent=percentile, sparse=False)
        adj_test = A_binarize(A_matrix=test_x, percent=percentile, sparse=False)
        # sparse matrix
    else:
        adj_train = deepcopy(train_x)
        adj_test = deepcopy(test_x)
    # consider part of the graph
    print("sparsity: ", scipy.sparse.issparse(adj_train[9]))  # check sparsity
    print("rank: ", np.linalg.matrix_rank(adj_train[9]))  # check matrix rank
    return adj_train, adj_test

In [10]:
run = 0
dropout_rate = 0.5
t_start = time.time()

print("Applying ICA...")
sec = 12  # Number of seconds in window?

train_x, test_x, y_train, y_test = preprocess_data(x_raw_all[:, 0], labels, run, Fs,
                                                    filt=False, ICA=True, A_Matrix='cov', sec=sec)

print("Creating brain graph....")
adj_train, adj_test = Adj_matrix(train_x, test_x)  # Creating functional connectivity graph

print("Preprocessing...")
# Compute number of nodes
num_nodes = adj_train.shape[1]

# If features are not used, replace feature matrix by identity matrix
I_train = np.tile(np.eye(adj_train.shape[1]), adj_train.shape[0]).T.reshape(-1, adj_train.shape[1],
                                                                            adj_train.shape[1])
I_test = np.tile(np.eye(adj_test.shape[1]), adj_test.shape[0]).T.reshape(-1, adj_test.shape[1], adj_test.shape[1])

features_train = np.ones((adj_train.shape[0], adj_train.shape[1], 1))

# Preprocessing on node features
num_features = features_train.shape[2]
features_nonzero = np.count_nonzero(features_train) // features_train.shape[0]

# Normalization and preprocessing on adjacency matrix
adj_norm_train = graph_norm(adj_train)
adj_label_train = adj_train + I_train

adj_norm_test = graph_norm(adj_test)
adj_label_test = adj_test + I_test

features_test = np.ones((adj_test.shape[0], adj_test.shape[1], 1))

train_dataset = (tf.data.Dataset.from_tensor_slices((adj_norm_train, adj_label_train, features_train))
                    .shuffle(len(adj_norm_train)).batch(64))
norm = adj_train.shape[1] * adj_train.shape[1] / float((adj_train.shape[1] * adj_train.shape[1]
                                                        - (adj_train.sum() / adj_train.shape[0])) * 2)


Applying ICA...
Start ICA
END ICA
ADF Statistic: -54.916290
p-value: 0.000000
Critical Values:
	1%: -3.431
	5%: -2.862
	10%: -2.567
Creating brain graph....
sparsity:  False
rank:  57
Preprocessing...


In [14]:
print(num_features, num_nodes, features_nonzero, norm)

1 57 57 0.666871921182266


In [36]:
Type = "float32"

class GraphConvolution(tf.keras.layers.Layer):
    """ Graph convolution layer """
    def __init__(self, input_dim, output_dim, num, act=tf.nn.relu, **kwargs):
        super().__init__(**kwargs)
        self.w = self.add_weight(name='weight' + str(num), shape=(input_dim, output_dim),
                             initializer="random_normal", trainable=True)
        self.b = self.add_weight(name='bias' + str(num),
                             initializer="zeros", shape=(output_dim,), trainable=True)
        self.act = act


    def call(self, inputs, adj, rate=0., normalize=False):
        x = tf.nn.dropout(inputs, rate=rate)
        x = tf.matmul(x, self.w)
        x = tf.matmul(adj, x)
        outputs = self.act(x + self.b)
        if normalize:
            x = tf.keras.utils.normalize(x)
        return outputs

class InnerProductDecoder(tf.keras.layers.Layer):
    """Symmetric inner product decoder layer"""

    def __init__(self, act=tf.nn.sigmoid, **kwargs):
        super().__init__(**kwargs)
        self.act = act

    def call(self, inputs, rate=0.):
        inputs = tf.nn.dropout(inputs, rate=rate)
        if (tf.shape(inputs).shape == 3):
            x = tf.transpose(inputs, perm=[0, 2, 1])
        else:
            x = tf.transpose(inputs)
        x = tf.matmul(inputs, x)
        outputs = self.act(x)
        return outputs

# Graph Variational Autoencoder
class GCNModelVAE(tf.keras.Model):
    def __init__(self, num_features, num_nodes, features_nonzero, **kwargs):
        super().__init__(**kwargs)
        self.input_dim = num_features
        self.features_nonzero = features_nonzero
        self.n_samples = num_nodes
        self.hidden_dim = 16  # hyperparameter
        self.embedding_dimension = 1  # hyperparameter
        self.hidden1 = GraphConvolution(input_dim=self.input_dim,
                                        output_dim=self.hidden_dim, num=1,
                                        act=lambda x: x, trainable=True)  # Convolutional layer
        self.hidden2 = GraphConvolution(input_dim=self.hidden_dim,
                                        output_dim=self.embedding_dimension * 2, num=2,
                                        act=lambda x: x, trainable=True)
        self.d = InnerProductDecoder(act=lambda x: x)
        self.d1 = GraphConvolution(input_dim=1,
                                   output_dim=self.n_samples, num=3,
                                   act=lambda x: x, trainable=True)  # Use new decoder model and loss function = 3???
    
    # Encoder feedforward
    def encoder(self, inputs, adj, rate):
        x = self.hidden1(inputs, adj=adj, rate=rate)
        x = self.hidden2(x, adj=adj, rate=rate)
        mean, logvar = tf.split(x, num_or_size_splits=2, axis=2)
        return mean, logvar

    # Reparametrization trick
    def reparameterize(self, mean, logvar):
        eps = tf.random.normal([self.n_samples, self.embedding_dimension])
        return eps * (tf.exp(logvar)) + mean

    # Decoding model
    def decoder(self, z, adj, rate=0., apply_sigmoid=False):
        logits = z
        logits = self.d(logits, rate=0.)
        feature = tf.ones((logits.shape[0], logits.shape[1], 1))
        logits = self.d1(feature, adj=logits, rate=rate)
        logits = tf.reshape(logits, [-1, self.n_samples * self.n_samples])
        if apply_sigmoid:
            probs = tf.sigmoid(logits)
            return probs
        return logits


lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=10000,
    decay_rate=0.9)  # learning rate


# VAE optimizer model
class OptimizerVAE(object):
    def __init__(self, model, num_nodes, num_features, norm):
        self.norm = norm
        self.num_nodes = num_nodes
        self.num_features = num_features
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

    def log_normal_pdf(self, sample, mean, logsd, raxis=[1, 2]):
        logvar = 2 * logsd
        log2pi = tf.math.log(2. * np.pi)
        out = tf.reduce_sum(-.5 * (tf.multiply((sample - mean) ** 2., tf.exp(-logvar)) + logvar + log2pi), axis=raxis)
        return out

    def bernoulli_log_density(self, logit, x):
        b = (x * 2) - 1
        return - tf.math.log(1 + tf.exp(-tf.multiply(b, logit)))

    def loss(self, y, x, adj, rate, model):
        mean, logvar = model.encoder(x, adj=adj, rate=rate)
        reparam = model.reparameterize(mean, logvar)
        reconstruct = model.decoder(reparam, adj, rate)

        
        preds_sub = tf.reshape(reconstruct, [-1, self.num_nodes, self.num_nodes])
        logpz = self.log_normal_pdf(reparam, 0., 0.)
        logqz_x = self.log_normal_pdf(reparam, mean, logvar)
        logpx_z = tf.reduce_sum(self.bernoulli_log_density(preds_sub, tf.cast(y, tf.float32)), [1, 2])
        return -tf.reduce_mean(logpx_z - ((logpz - logqz_x)))

    def loss2(self, y, x, adj, rate, model):
        mean, logvar = model.encoder(x, adj, rate)
        reparam = model.reparameterize(mean, logvar)
        reconstruct = model.decoder(reparam, adj, rate)
        preds_sub = tf.reshape(reconstruct, [-1, self.num_nodes, self.num_nodes])
        cost = self.norm * tf.reduce_mean(tf.reduce_sum(
            tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.cast(y, tf.float32),
                                                    logits=preds_sub), [1, 2]))
        kl = (0.5 / self.num_nodes) * \
             tf.reduce_mean(tf.reduce_sum(1 \
                                          + 2 * logvar \
                                          - tf.square(mean) \
                                          - tf.square(tf.exp(logvar)), [1, 2]))
        cost -= kl
        return cost

    def train_step(self, y, x, adj, rate, model):
        with tf.GradientTape() as tape:
            cost = self.loss(y, x, adj=adj, rate=rate, model=model)
        assert not np.any(np.isnan(cost.numpy()))
        gradients = tape.gradient(cost, model.trainable_variables)
        opt_op = self.optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        return cost

In [28]:
print("Initializing...")
# VAE model
GVAE_model = GCNModelVAE(num_features, num_nodes, features_nonzero)
# Optimizer
optimizer = OptimizerVAE(model=GVAE_model, num_nodes=num_nodes,
                            num_features=num_features, norm=norm)

Initializing...


In [30]:
GVAE_model.trainable_variables

[<KerasVariable shape=(1, 16), dtype=float32, path=graph_convolution_22/weight1>,
 <KerasVariable shape=(16,), dtype=float32, path=graph_convolution_22/bias1>,
 <KerasVariable shape=(16, 2), dtype=float32, path=graph_convolution_23/weight2>,
 <KerasVariable shape=(2,), dtype=float32, path=graph_convolution_23/bias2>,
 <KerasVariable shape=(1, 57), dtype=float32, path=graph_convolution_24/weight3>,
 <KerasVariable shape=(57,), dtype=float32, path=graph_convolution_24/bias3>]

In [37]:
print("Initializing...")
# VAE model
GVAE_model = GCNModelVAE(num_features, num_nodes, features_nonzero)
# Optimizer
optimizer = OptimizerVAE(model=GVAE_model, num_nodes=num_nodes,
                            num_features=num_features, norm=norm)



print("Training...")

prev_cost = float("inf")
stop_val = 0
stop_num = 10
FLAGS_shuffle = False
for epoch in range(1000):
    t = time.time()
    # Compute average loss
    loss = 0
    for adj, label, x in train_dataset:
        # print(label, tf.cast(x, tf.float32),  tf.cast(adj, tf.float32))
        loss += optimizer.train_step(label, 
                                     tf.cast(x, tf.float32), 
                                     tf.cast(adj, tf.float32), 
                                     dropout_rate, GVAE_model)

    avg_cost = loss.numpy() / (len(adj_train))

    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(round(avg_cost, 3)),
            "time=", "{:.5f}".format(round(time.time() - t, 3)))
    Computational_time[run] += (time.time() - t)
    num_epoch[run] += 1

    # When to stop the iteration
    if (prev_cost < avg_cost):
        stop_val += 1
        if (stop_val == stop_num):
            break
    else:
        stop_val = 0
        prev_cost = avg_cost

Computational_time[run] = Computational_time[run] / num_epoch[run]

print("Time for each epoch:", np.round(Computational_time[run], 3))

meanr, logvarr = GVAE_model.encoder(tf.cast(features_train, tf.float32), tf.cast(adj_norm_train, tf.float32), 0.)
ztr = GVAE_model.reparameterize(meanr, logvarr)
meane, logvare = GVAE_model.encoder(tf.cast(features_test, tf.float32), tf.cast(adj_norm_test, tf.float32), 0.)
zte = GVAE_model.reparameterize(meane, logvare)

train_feature = deepcopy(ztr).numpy().reshape(len(ztr), -1)
test_feature = deepcopy(zte).numpy().reshape(len(zte), -1)


Initializing...
Training...
Epoch: 0001 train_loss= 37.90500 time= 0.72000
Epoch: 0002 train_loss= 37.08800 time= 0.70700
Epoch: 0003 train_loss= 36.96200 time= 0.71800
Epoch: 0004 train_loss= 37.10700 time= 0.70100
Epoch: 0005 train_loss= 37.68900 time= 0.69700
Epoch: 0006 train_loss= 37.13800 time= 0.68100
Epoch: 0007 train_loss= 36.94400 time= 0.67900
Epoch: 0008 train_loss= 36.84300 time= 0.69300
Epoch: 0009 train_loss= 36.55700 time= 0.68100
Epoch: 0010 train_loss= 36.69500 time= 0.68400
Epoch: 0011 train_loss= 36.58500 time= 0.71600
Epoch: 0012 train_loss= 36.53100 time= 0.70400
Epoch: 0013 train_loss= 36.46200 time= 0.69300
Epoch: 0014 train_loss= 36.36700 time= 0.69900
Epoch: 0015 train_loss= 36.31200 time= 0.68000
Epoch: 0016 train_loss= 36.39300 time= 0.69600
Epoch: 0017 train_loss= 36.10600 time= 0.69100
Epoch: 0018 train_loss= 36.13200 time= 0.69400
Epoch: 0019 train_loss= 35.98600 time= 0.67200
Epoch: 0020 train_loss= 36.04000 time= 0.67200
Epoch: 0021 train_loss= 35.90700

AssertionError: 